## Introduction

In this notebook, we will explore the process of gathering and preprocessing historical daily stock prices and commodities prices, as well as macroeconomic and technical analysis indicators. This data will be used to train several commonly used regression models to predict the variation of the price of a stock for the next day. We will analyze the performance of multiple linear, decision tree, random forest, support vector, xgbost, and catbost regression models, optimized using Bayesian optimization techniques. Additionally, we will validate each model using the time series hold one out cross-validation strategy, and quantify and graph their performances.

## Gathering and Preprocessing Data

In [1]:
# If you don't have them installed here are the packages we will be using
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install sklearn
# !pip install tensorflow #
# !pip install yfinance
# !pip install plotly #
# !pip install pandas_ta #
# !pip install scikit-optimize # for hyperparameter tuning
# !pip install TA-Lib # for technical analysis
# !pip install xgboost # 
# !pip install catboost # 

In [2]:
# Import used libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import pandas_ta as ta
import warnings
warnings.filterwarnings('ignore')

sns.set()

In [3]:
# Import Petrobras data from Yahoo Finance trough pandas_ta
df = pd.DataFrame()
df = df.ta.ticker("petr4.sa", start='1998-01-01', end='2022-12-27')
df.index = df.index.date
df.index.name = 'Date'
# make index datetime
df.index = pd.to_datetime(df.index)
# sort by index
df = df.sort_index()

# get days in which stocks were traded (aka: the index the df)
days_total = (pd.DataFrame(df.index)
                .reset_index(drop=False)
                .set_index('Date')
                .drop(columns=['index']))

In [4]:
## functions
def apply_delay(s, forwarddelay):
    s_copy = s.copy()
    s_copy = s_copy.shift(forwarddelay)
    return s_copy

# adder of a few lags to the dataframe
def get_data_w_some_lags(df):
    df = df.copy()

    # Create columns of returns of Open, High, Low, Close
    for i in range(4):
        df[df.columns[i] + '_pct_change'] = df[df.columns[i]].pct_change(fill_method=None)

    # forwarddelay = 1, 2
    for lag in [1, 2]:
        for col in ['Open', 'High', 'Low', 'Close']:
            df[col + '_pct_change_lag_f' + str(lag)] = apply_delay(df[col + '_pct_change'], lag)

    # make the return of the last lag days a feature
    for lag in [2, 5, 10]:
        df['Close_cum_pct_change_from_' + str(lag)] = df['Close'].pct_change(periods=lag, fill_method=None)

    # make the weekly return a feature for the last 3 weeks (the first we already have)
    for lag in [2, 3]:
        df['Close_cum_pct_change_from_' + str(lag) + '_week'] = (df['Close'].shift((lag-1)*5)
                                                               / df['Close'].shift(lag*5)
                                                               -1)
    
    # make the monthly return a feature for the last 2 months
    for lag in [1, 2]:
        df['Close_cum_pct_change_from_' + str(lag) + '_month'] = (df['Close'].shift((lag-1)*20)
                                                                / df['Close'].shift(lag*20)
                                                                - 1)

    # make sma of 3 days for 'Volume' as a feature
    if 'Volume' in df.columns:
        df['Volume_sma_3'] = df['Volume'].rolling(3).mean()

    return df

# adder of lags to the dataframe
def get_data_w_lags(df):
    df = df.copy()

    # Create columns of % change of Open, High, Low, Close
    for i in range(4):
        df[df.columns[i] + '_pct_change'] = df[df.columns[i]].pct_change(fill_method=None)

    # forwarddelay = 1, 2, 3, 5, 10, 15, 20
    for lag in [1, 2, 3, 5, 10, 15, 20]:
        for col in ['Open', 'High', 'Low', 'Close']:
            df[col + '_pct_change_lag_f' + str(lag)] = apply_delay(df[col + '_pct_change'], lag)

    # make the return of the last lag days a feature
    for lag in [2, 3, 5, 10, 15, 20]:
        df['Close_cum_pct_change_from_' + str(lag)] = df['Close'].pct_change(periods=lag, fill_method=None)

    # make the weekly return a feature for the last 4 weeks (the first we already have)
    for lag in [2, 3, 4]:
        df['Close_cum_pct_change_from_' + str(lag) + '_week'] = (df['Close'].shift((lag-1)*5)
                                                               / df['Close'].shift(lag*5)
                                                               -1)
    
    # make the monthly return a feature for the last 3 months
    for lag in [1, 2]:
        df['Close_cum_pct_change_from_' + str(lag) + '_month'] = (df['Close'].shift((lag-1)*20)
                                                                / df['Close'].shift(lag*20)
                                                                - 1)

    # make sma of 3 days for 'Volume' as a feature
    if 'Volume' in df.columns:
        df['Volume_sma_3'] = df['Volume'].rolling(3).mean()

    return df

# add the time lags to the benchmark dataframe
def add_time_lags_to_bench(df_bench, days):
    df_bench = (days.loc['2000-01-31':]
                    .merge(df_bench, how='left', left_index=True, right_index=True)
                    .ffill())
    
    # take Open, High, Low, and Close columns, apply get_data_w_lags function and concatenate the dataframe
    df_bench_enlarged = pd.DataFrame()
    for col in set([c.split('_')[0] for c in df_bench.columns]):

        df = (df_bench.filter(regex=col)
                      .rename(columns={f'{col}_Close': 'Close',
                                       f'{col}_Open': 'Open',
                                       f'{col}_High': 'High',
                                       f'{col}_Low': 'Low'}))
        if col == 'IBOV':
            df = get_data_w_lags(df)
            df.columns = [f'{col}_{c}' for c in df.columns]
        else:
            df = get_data_w_some_lags(df)
            if col == 'XAU':
                df.columns = [f'GOLD_{c}' for c in df.columns]
            else:
                df.columns = [f'USDBRL_{c}' for c in df.columns]
        df_bench_enlarged = pd.concat([df_bench_enlarged, df], axis=1)
    return df_bench_enlarged.loc['2001-01-31':], days.loc['2001-01-31':]


In [5]:
## prepare external features
# all data from 2001-01-31 to 2020-12-31
df_macro = pd.read_csv('data_raw/Dados_MacroBR_Historico.csv', sep=',', index_col=0, parse_dates=True)
df_comm = pd.read_csv('data_raw/Dados_Commodities_Historico.csv', sep=',', index_col=0, parse_dates=True)
df_bench = pd.read_csv('data_raw/Dados_Ibov_Dolar_Ouro.csv', sep=',', index_col=0, parse_dates=True)

# expand the temporal caracteristics of the benchmark dataframe as new features and make it start at '2001-01-31'
df_bench_enlarged, days = add_time_lags_to_bench(df_bench, days_total)
df_macro.loc[:'2001-01-31'] = df_macro.loc[:'2001-01-31'].ffill()
columns = df_macro.columns[df_macro.loc['2001-01-31'].isna()].tolist()
df_macro = df_macro.drop(columns=columns)

# merge all dataframes and fix them to the same index (dates in which the market has opened)
df_merge = (days.merge(df_macro, how='left', left_index=True, right_index=True)
                .merge(df_comm, how='left', left_index=True, right_index=True)
                .merge(df_bench_enlarged, how='left', left_index=True, right_index=True))

# fill nan values with the last value
df_merge = df_merge.ffill()


In [6]:
# check the range of the data
last_date = np.min([df_merge[col].last_valid_index() for col in df_merge.columns])
first_date = np.max([df_merge[col].first_valid_index() for col in df_merge.columns])
print(f'First date: {first_date}\nLast date: {last_date}')

First date: 2001-01-31 00:00:00
Last date: 2022-12-26 00:00:00


In [7]:
## functions
# input economic rules as features
def input_classes(df):
    df['AO_5_34_dir'] = [1 if x > 0 else 0 for x in df['AO_5_34']]
    df['BOP_dir'] = [1 if x > 0 else 0 for x in df['BOP']]
    df['CCI_14_0.015_dir'] = [1 if x > -100 else 0 for x in df['CCI_14_0.015']]
    df['CFO_9_dir'] = [1 if x > 0 else 0 for x in df['CFO_9']]
    df['CG_10_dir'] = [1 if x > df['CG_10'].shift(1)[i] else 0 for i, x in enumerate(df['CG_10'])]
    df['COPC_11_14_10_dir'] = [1 if (x < 0 and x > df['COPC_11_14_10'][i-1]) else 0 for i, x in enumerate(df['COPC_11_14_10'])]
    df['CTI_12_dir'] = [1 if x > 0 else 0 for x in df['CTI_12']]
    df['DM_14_dir'] = [1 if df['DMP_14'][i] > df['DMN_14'][i] else 0 for i in range(len(df))]
    df['FISHERT_9_1_dir'] = [1 if df['FISHERT_9_1'][i] > df['FISHERTs_9_1'][i] else 0 for i in range(len(df))]
    df['INERTIA_20_14_dir'] = [1 if x > 50 else 0 for x in df['INERTIA_20_14']]
    df['KDJ_9_3_dir'] = [1 if (df['K_9_3'][i] > df['D_9_3'][i] and df['K_9_3'][i] > 20 and df['K_9_3'][i] < 80) else 0 for i in range(len(df))]
    df['KST_10_15_20_30_10_10_10_15_dir'] = [1 if (df['K_9_3'][i] > df['D_9_3'][i] and df['K_9_3'][i] > 20 and df['K_9_3'][i] < 80) else 0 for i in range(len(df))]
    df['MACD_12_26_9_dir'] = [1 if x > 0 else 0 for x in df['MACDh_12_26_9']]
    df['MOM_10_dir'] = [1 if x > 0 else 0 for x in df['MOM_10']]
    df['PGO_14_dir'] = [1 if x > 0 else 0 for x in df['PGO_14']]
    df['PSL_12_dir'] = [1 if x > 50 else 0 for x in df['PSL_12']]
    df['ROC_10_dir'] = [1 if x > 0 else 0 for x in df['ROC_10']]
    df['RSI_14_dir'] = [1 if (x > 30 and x < 70) else 0 for x in df['RSI_14']]
    df['RSX_14_dir'] = [1 if (x > 30 and x < 70) else 0 for x in df['RSX_14']]
    df['RVGI_14_4_dir'] = [1 if df['RVGI_14_4'][i] > df['RVGIs_14_4'][i] else 0 for i in range(len(df))]
    df['SMI_5_20_5_dir'] = [1 if x > 0 else 0 for x in df['SMI_5_20_5']]
    df['STOCH_14_3_3_dir'] = [1 if df['STOCHk_14_3_3'][i] > df['STOCHd_14_3_3'][i] else 0 for i in range(len(df))]
    df['UO_7_14_28_dir'] = [1 if (x > 30 and x < 70) else 0 for x in df['UO_7_14_28']]
    df['WILLR_14_dir'] = [1 if (x > -80 and x < -20) else 0 for x in df['WILLR_14']]
    df['ADX_14_dir'] = [1 if x > 30 else 0 for x in df['ADX_14']]
    df['AROON_14_dir'] = [1 if df['AROONU_14'][i] > df['AROOND_14'][i] else 0 for i in range(len(df))]
    df['CKSP_10_3_20_dir'] = [1 if (df['Close'][i] > df['CKSPl_10_3_20'][i] and df['Close'][i] > df['CKSPs_10_3_20'][i]) else 0 for i in range(len(df))]
    df['QS_10_dir'] = [1 if x > 0 else 0 for x in df['QS_10']]
    df['VHF_28_dir'] = [1 if x > 0.5 else 0.5 for x in df['VHF_28']]
    df['VTX_14_dir'] = [1 if df['VTXP_14'][i] > df['VTXM_14'][i] else 0 for i in range(len(df))]
    df['MCGD_10_dir'] = [1 if df['Close'][i] > df['MCGD_10'][i] else 0 for i in range(len(df))]
    df['VIDYA_14_dir'] = [1 if df['Close'][i] > df['VIDYA_14'][i] else 0 for i in range(len(df))]
    return df

# feature engineering function
def feature_engineering(df):
    df = df.copy()
    
    # get time lags
    df = get_data_w_lags(df)

    # get technical indicators from the pandas_ta library as features
    df.ta.strategy()

    # drop the columns that have look-ahead bias or that yeild to many NaNs
    df = df.drop(['DPO_20','ICS_26','HILOl_13_21','HILOs_13_21','ISB_26','KVOs_34_55_13','VWAP_D',
                    'PSARl_0.02_0.2','PSARs_0.02_0.2','QQE_14_5_4.236','QQEl_14_5_4.236','EOM_14_100000000',
                    'QQEs_14_5_4.236','SUPERTl_7_3.0','SUPERTs_7_3.0','TRIX_30_9','TSIs_13_25_13',
                    'TRIXs_30_9','ISA_9','KSTs_9','KVO_34_55_13','T3_10_0.7'], axis=1)

    # the first 2 months have the majority of the missing values, so we will drop them
    df = (df.drop(df.index[:46])
            .drop(df.index[-1]))

    # # gather the columns that have more than 1% of the data missing and print them to file
    # columns_droped = df.columns[df.isna().sum(axis=0) > df.shape[0]*0.01]
    # with open('columns_droped.txt', 'a') as f:
    #     f.write(str(columns_droped))

    df = input_classes(df)

    return df

# fill the infinities of each column with the maximum value of that column
def fill_infinities(df):
    n_infinities = df.isin([np.inf, -np.inf]).sum(axis=0).sum()
    # loop through the columns with infinities
    for col in df.columns[df.isin([np.inf, -np.inf]).sum(axis=0) > 0]:
        # get the maximum value which is not infinity of the column
        max_value = df[col].replace([np.inf, -np.inf], np.nan).max()
        min_value = df[col].replace([np.inf, -np.inf], np.nan).min()
        # replace the infinities with the maximum value
        df[col].replace([np.inf], max_value, inplace=True)
        df[col].replace([-np.inf], min_value, inplace=True)
    return df, n_infinities

# clip outliers by capping and flooring the values to the 4 std from the mean
def clip_outliers(df):
    n_outliers = {}
    for col in df.columns:
        # get the mean and std of the column
        mean = df[col].mean()
        std = df[col].std()
        # get the number of outliers
        if df[col][df[col] > mean + 4*std].count() + df[col][df[col] < mean - 4*std].count() > 0:
            n_outliers[col] = df[col][df[col] > mean + 4*std].count() + df[col][df[col] < mean - 4*std].count()
        # cap the outliers
        df[col] = df[col].clip(mean - 4*std, mean + 4*std)
    # sort the outliers by the number of outliers
    n_outliers = {k: v for k, v in sorted(n_outliers.items(), key=lambda item: item[1], reverse=True)}
    return df, n_outliers

In [8]:
# add the features to df_ticker
df_w_features_all = feature_engineering(df)
df_w_features_all = df_w_features_all.merge(df_merge, how='left', left_index=True, right_index=True)#.reset_index()
df_w_features = df_w_features_all.loc['2001-01-31':'2022-12-27'].copy()
if df_w_features.isna().sum().sum() > 0:
    print(f'There are still {df_w_features.isna().sum().sum() > 0} missing values in the dataframe')
    df_w_features.isna().sum(axis=1).sort_values(ascending=False)
    df_w_features.isna().sum(axis=0).sort_values(ascending=False)
df_w_features.shape

# make target variable
df_w_features['target'] = df_w_features['Close'].pct_change()

# define macros for the columns
COL_TO_CLIP = [str(col) for col in df_w_features.columns if col not in ['Date', 'target', 'Close_pct_change']]
MOVE_FORWARD = [str(col) for col in df_w_features.columns if col not in ['Date', 'target']]
FEATURES = [str(col) for col in df_w_features.columns if col not in ['Date', 'target']]
TARGET = 'target'

# shift the features one day forward so that each row has the features of the previous day and the target of the current day
df_w_features.loc[:, MOVE_FORWARD] = df_w_features.loc[:, MOVE_FORWARD].shift(1)
# with open('columns_droped.txt', 'a') as f:
#     f.write(str([str(col) for col in df_w_features.columns]))
df_w_features = df_w_features.dropna().reset_index()

# apply the fill_infinities function to the train and test data
df_w_features, n_infinities = fill_infinities(df_w_features)
print(f"{n_infinities} infinities filled in the data.")

# apply the fill_outliers function to the train and test data for the columns after the 'Close_pct_change_lag_b1' column (pandas_ta columns)
df_w_features.loc[:, COL_TO_CLIP], n_outliers = clip_outliers(df_w_features.loc[:, COL_TO_CLIP])
print(f"Outliers cliped by column: {n_outliers}.")

4 infinities filled in the data.
Outliers cliped by column: {'CDL_MARUBOZU': 273, 'CDL_3OUTSIDE': 224, 'BZGDQOQ%': 185, 'CDL_HARAMICROSS': 144, 'CDL_HIKKAKE': 133, 'CDL_HANGINGMAN': 124, 'CDL_3INSIDE': 120, 'CDL_DOJISTAR': 117, 'CDL_HAMMER': 104, 'CDL_MATCHINGLOW': 102, 'CDL_HOMINGPIGEON': 95, 'PVOL': 91, 'CDL_SHOOTINGSTAR': 84, 'CDL_INVERTEDHAMMER': 81, 'Volume_sma_3': 78, 'CDL_TAKURI': 77, 'CDL_DRAGONFLYDOJI': 74, 'Volume': 72, 'MAD_30': 66, 'EFI_13': 65, 'BZPIIPMO': 64, 'BZRTRETM': 64, 'STDEV_30': 63, 'VAR_30': 63, 'ADOSC_3_10': 62, 'DMN_14': 62, 'PVOs_12_26_9': 62, 'UI_14': 62, 'CDL_GRAVESTONEDOJI': 61, 'MOM_10': 58, 'SQZ_20_2.0_20_1.5': 58, 'SQZPRO_20_2.0_20_2_1.5_1': 58, 'AO_5_34': 56, 'MACDh_12_26_9': 56, 'PVO_12_26_9': 56, 'APO_12_26': 54, 'MEDIAN_30': 53, 'QTL_30_0.5': 53, 'BCOMNI': 51, 'BEARP_13': 49, 'MACD_12_26_9': 49, 'MACDs_12_26_9': 49, 'STCmacd_10_12_26_0.5': 49, 'ABER_ATR_5_15': 48, 'SLOPE_1': 47, 'ADX_14': 45, 'ATRr_14': 44, 'IKS_26': 44, 'DCU_20_20': 43, 'BZPIIPCM': 

## Regressions

In [9]:
# import time series split
from sklearn.model_selection import TimeSeriesSplit

# import bayesian optimization libraries
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_objective, plot_histogram

# import preprocessing libs
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel

X = df_w_features[FEATURES]
y = df_w_features[TARGET]

# DataFrame to store the results
df_results = pd.DataFrame()
df_results['y_test'] = y[-252:]

# DataFrame to store the feature importances of each model
df_feature_importances = pd.DataFrame(index=FEATURES)

# Save model
def save_objet(obj, filename):
    import joblib
    path_lst = joblib.dump(obj, filename)
    for i in range(len(path_lst)):
        print(f"Object saved in {path_lst[i]}")

# Plot the hyperparameter tuning results
def tuning_plots(bayes_search, param_dist):
    for i in range(len(bayes_search.optimizer_results_)):
        if len(bayes_search.optimizer_results_[i].models) == 0:
            continue
        # Plot the objective function
        plot_objective(bayes_search.optimizer_results_[i])
        plt.show()

        # Plot the hyperparameter chosen values
        fig, axes = plt.subplots(1, len(param_dist[i]), figsize=(15, 5))
        for j, ax in enumerate(axes):
            plot_histogram(bayes_search.optimizer_results_[i], j, ax=ax)
        plt.show()

def print_scores(bayes_search, test_index):
    # Print the best parameters and the best score
    print("Best parameters: ", bayes_search.best_params_)
    print("Best Accuracy: {:.2f} %".format(bayes_search.best_score_*100))

    # get the r2 score of the best model
    from sklearn.metrics import r2_score
    best_model = bayes_search.best_estimator_
    print("Best Model R2 score: {:.4f}".format(
                                        r2_score(
                                        y.loc[list(test_index)],
                                        best_model.predict(X.loc[list(test_index)]))))

In [10]:
# Multiple Linear Regression
def multiple_linear_regression(X, y, df_results, df_feature_importances, verbose=False):

    # import regression model
    from sklearn.linear_model import LinearRegression

    tss = TimeSeriesSplit(n_splits=5, test_size=252*1, gap=1)

    #make pipeline
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA()),
        ('regressor', LinearRegression())
        ])

    # Define the hyperparameters to be tuned using BayesSearchCV's search space object
    param_dist = [{
        'pca__n_components': Real(0.3, 0.8),
        'regressor__fit_intercept': Categorical([True, False]),
        'regressor__positive': Categorical([True, False])
    }]

    # Create the BayesSearchCV object
    bayes_search = BayesSearchCV(pipe, param_dist, n_iter=25, scoring='r2', cv=tss, n_jobs=-1)

    # Fit the BayesSearchCV object to the data
    bayes_search.fit(X[:-252], y[:-252])

    # get the last train/test split indexes
    for _, test_index in tss.split(X):
        pass

    # Predicting the Test set results
    df_results['mlr_y_pred'] = bayes_search.best_estimator_.predict(X.loc[list(test_index)])

    print_scores(bayes_search, test_index)
    if verbose:
        tuning_plots(bayes_search, param_dist)
        display(pd.DataFrame(bayes_search.cv_results_))
    save_objet(bayes_search, 'saved_models/fitted_pipe_mlr.joblib')
    # df_feature_importances['mlr'] = bayes_search.best_estimator_.named_steps['regressor'].coef_

    return df_results

df_results = multiple_linear_regression(X, y, df_results, df_feature_importances)

# first training results
# Best parameters:  {'fit_intercept': False, 'positive': True}
# Best Accuracy: -58.67 %
# Best Model R2 score: -0.0234
# Best parameters:  {'pca__n_components': 44, 'regressor__fit_intercept': True, 'regressor__positive': True}
# Best Accuracy: -1.77 %
# Best Model R2 score: -0.0169
# Best parameters:  {'pca__n_components': 0.76, 'regressor__fit_intercept': True, 'regressor__positive': True}
# Best Accuracy: -2.84 %
# Best Model R2 score: -0.0111
## with commodities, exchange rates, and bench (ibov)
# Best parameters:  OrderedDict([('pca__n_components', 0.4269376887065359), ('regressor__fit_intercept', True), ('regressor__positive', True)])
# Best Accuracy: -0.09 %
# Best Model R2 score: -0.0021

Best parameters:  OrderedDict([('pca__n_components', 0.3657244526385898), ('regressor__fit_intercept', True), ('regressor__positive', True)])
Best Accuracy: -0.05 %
Best Model R2 score: -0.0024
Object saved in saved_models/fitted_pipe_mlr.joblib


In [11]:
# Decision Tree Regression
def deciosion_tree_regression(X, y, df_results, df_feature_importances, verbose=False):

    # import regression model
    from sklearn.tree import DecisionTreeRegressor

    tss = TimeSeriesSplit(n_splits=5, test_size=252*1, gap=1)

    # Create the pipeline
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('dim_reduction', PCA()),
        ('feature_selection', SelectFromModel(DecisionTreeRegressor())),
        ('regression', DecisionTreeRegressor(random_state = 0))
    ])

    # Define the hyperparameters to be tuned
    param_dist = [
        {
        'dim_reduction__n_components': Real(0.3, 0.8),
        'regression__max_depth': Integer(1, 5),
        'regression__min_samples_leaf': Integer(1, 35)
        },
        {
        'feature_selection__estimator__max_depth': Integer(1, 5),
        'feature_selection__estimator__min_samples_leaf': Integer(1, 25),
        'feature_selection__threshold': Categorical(['0.2*mean', '1*mean', '2*mean', '3*mean']),
        'regression__max_depth': Integer(1, 5),
        'regression__min_samples_leaf': Integer(1, 35)
        }
    ]

    # Create the BayesSearchCV object
    bayes_search = BayesSearchCV(pipe, param_dist, n_iter=25, scoring='r2', cv=tss, n_jobs=-1)

    # Fit the BayesSearchCV object to the data
    bayes_search.fit(X[:-252], y[:-252])

    # get the last train/test split indexes
    for _, test_index in tss.split(X):
        pass

    # Predicting the Test set results
    df_results['dtr_y_pred'] = bayes_search.best_estimator_.predict(X.loc[list(test_index)])

    print_scores(bayes_search, test_index)
    if verbose:
        tuning_plots(bayes_search, param_dist)
        display(pd.DataFrame(bayes_search.cv_results_))
    save_objet(bayes_search, 'saved_models/fitted_pipe_dtr.joblib')
    # print(bayes_search.best_estimator_.named_steps['regression'].feature_importances_)

    return df_results

df_results = deciosion_tree_regression(X, y, df_results, df_feature_importances)

# first training results
## with commodities, exchange rates, and bench (ibov)
# Best parameters:  OrderedDict([('dim_reduction__n_components', 0.47854272610088305), ('regression__max_depth', 1), ('regression__min_samples_leaf', 25)])
# Best Accuracy: 0.21 %
# Best Model R2 score: 0.0064
# Model saved in saved_models/fitted_pipe_dtr.joblib

Best parameters:  OrderedDict([('feature_selection__estimator__max_depth', 1), ('feature_selection__estimator__min_samples_leaf', 25), ('feature_selection__threshold', '3*mean'), ('regression__max_depth', 1), ('regression__min_samples_leaf', 13)])
Best Accuracy: 0.24 %
Best Model R2 score: -0.0940
Object saved in saved_models/fitted_pipe_dtr.joblib


In [12]:
# Random Forest Regression
def random_forest_regression(X, y, df_results, df_feature_importances, verbose=False):

    # import regression model
    from sklearn.ensemble import RandomForestRegressor

    tss = TimeSeriesSplit(n_splits=5, test_size=252*1, gap=1)

    # Create the pipeline
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('dim_reduction', PCA()),
        ('feature_selection', SelectFromModel(RandomForestRegressor(n_estimators=10, random_state=0))),
        ('regression', RandomForestRegressor(n_estimators=25, random_state=0))
    ])

    # Define the hyperparameters to be tuned
    param_dist = [{
        'dim_reduction__n_components': Real(0.3, 0.9),
        'regression__max_depth': Integer(1, 3),
        'regression__min_samples_leaf': Integer(5, 35)
    },
    {
        'feature_selection__estimator__max_depth': Integer(1, 3),
        'feature_selection__estimator__min_samples_leaf': Integer(5, 35),
        'feature_selection__threshold': Categorical(['0.2*mean', '1*mean', '2*mean', '3*mean']),
        'regression__max_depth': Integer(1, 3),
        'regression__min_samples_leaf': Integer(5, 35)
    }]

    # Create the BayesSearchCV object
    bayes_search = BayesSearchCV(pipe, param_dist, n_iter=25, scoring='r2', cv=tss, n_jobs=-1)

    # Fit the BayesSearchCV object to the data
    bayes_search.fit(X[:-252], y[:-252])

    # get the last train/test split indexes
    for _, test_index in tss.split(X):
        pass

    # Predicting the Test set results
    df_results['rfr_y_pred'] = bayes_search.best_estimator_.predict(X.loc[list(test_index)])

    print_scores(bayes_search, test_index)
    if verbose:
        tuning_plots(bayes_search, param_dist)
        display(pd.DataFrame(bayes_search.cv_results_))
    save_objet(bayes_search, 'saved_models/fitted_pipe_rfr.joblib')

    # store feature importance
    # loop over the features used in the best model and add the feature importance to the dataframe
    # for i, feature in enumerate(bayes_search.best_estimator_.named_steps['feature_selection'].get_support(indices=True)):
    #     df_feature_importances.loc[df_feature_importances.index[feature], 'rfr'] = bayes_search.best_estimator_.named_steps['regression'].feature_importances_[i]
    # df_feature_importances.fillna(0, inplace=True)

    return df_results

df_results = random_forest_regression(X, y, df_results, df_feature_importances)

# Best parameters:  {'dim_reduction__n_components': 0.74, 'regression__max_depth': 1, 'regression__max_features': 1.0, 'regression__min_samples_leaf': 29, 'regression__n_estimators': 10}
# Best Accuracy: 0.04 %
# Best Model R2 score: -0.0015
## with commodities, exchange rates, and bench (ibov)
# Best parameters:  OrderedDict([('dim_reduction__n_components', 0.8), ('regression__max_depth', 1), ('regression__min_samples_leaf', 19)])
# Best Accuracy: 0.01 %
# Best Model R2 score: 0.0009
# Model saved in saved_models/fitted_pipe_rtr.joblib

Best parameters:  OrderedDict([('dim_reduction__n_components', 0.8), ('regression__max_depth', 1), ('regression__min_samples_leaf', 18)])
Best Accuracy: 0.34 %
Best Model R2 score: -0.0043
Object saved in saved_models/fitted_pipe_rfr.joblib


In [13]:
# Support Vector Regression (SVR)
def suport_vector_regression(X, y, df_results, verbose=False):

    from sklearn.svm import SVR, LinearSVR

    tss = TimeSeriesSplit(n_splits=5, test_size=252*1, gap=1)

    # Create the pipeline
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('dim_reduction', PCA()),
        ('feature_selection', SelectFromModel(SVR(kernel='linear'))),
        ('regression', SVR())
    ])

    # Define the hyperparameters to be tuned
    param_dist = [
        {
        'regression': [LinearSVR(max_iter=3000)],
        'regression__C': Real(1e-6, 1e+1, 'log-uniform'),
        },
        {
        'dim_reduction__n_components': Real(0.3, 0.9),
        'regression__kernel': Categorical(['linear']),
        'regression__C': Real(1e-6, 1, 'log-uniform')
        },
        {
        'feature_selection__threshold': Categorical(['0.2*mean', '1*mean', '2*mean', '3*mean']),
        'feature_selection__estimator__kernel': Categorical(['linear']),
        'feature_selection__estimator__C': Real(1e-6, 1, 'log-uniform'),
        'regression__kernel': Categorical(['linear']),
        'regression__C': Real(1e-6, 1, 'log-uniform')
        },
        {
        'dim_reduction__n_components': Real(0.3, 0.9),
        'regression__kernel': Categorical(['rbf', 'sigmoid']),
        'regression__gamma': Real(1e-6, 1e+1, 'log-uniform'),
        'regression__C': Real(1e-6, 1, 'log-uniform'),
        'regression__coef0': Real(-1, 1)
        }]

    # Create the BayesSearchCV object
    bayes_search = BayesSearchCV(pipe, param_dist, n_iter=25, scoring='r2', cv=tss, n_jobs=-1)

    # Fit the BayesSearchCV object to the data
    bayes_search.fit(X[:-252], y[:-252])

    # get the last train/test split indexes
    for _, test_index in tss.split(X):
        pass

    # Predicting the Test set results
    df_results['svr_y_pred'] = bayes_search.best_estimator_.predict(X.loc[list(test_index)])

    print_scores(bayes_search, test_index)
    if verbose:
        tuning_plots(bayes_search, param_dist)
        display(pd.DataFrame(bayes_search.cv_results_))
    save_objet(bayes_search, 'saved_models/fitted_pipe_svr.joblib')

    return df_results

df_results = suport_vector_regression(X, y, df_results)

# first training results
# Best parameters:  {'dim_reduction__n_components': 0.8, 'regression__C': 1e-06, 'regression__kernel': 'linear'}
# Best Accuracy: -0.71 %
# Best Model R2 score: 0.0000

/home/vfranco-/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/vfranco-/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/vfranco-/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/vfranco-/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/vfranco-/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/vfranco-/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinea

Best parameters:  OrderedDict([('regression', LinearSVR(C=1e-06, max_iter=3000)), ('regression__C', 1e-06)])
Best Accuracy: -0.33 %
Best Model R2 score: -0.0074
Object saved in saved_models/fitted_pipe_svr.joblib


In [14]:
# XGBoost
def XGBoost(X, y, df_results, df_feature_importances, verbose=False):

    from xgboost import XGBRegressor

    tss = TimeSeriesSplit(n_splits=5, test_size=252*1, gap=1)

    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA()),
        ('feature_selection', SelectFromModel(XGBRegressor(n_estimators=10, random_state=0))),
        ('regression', XGBRegressor(random_state=0))
    ])

    # define the hyperparameters to be tuned
    param_dist = [{
        'pca__n_components': Real(0.3, 0.9),
        'regression__booster': Categorical(['gbtree']),
        'regression__max_depth': Integer(1, 5),
        'regression__learning_rate': Real(1e-4, 3e-1, 'log-uniform')
        }]

    # Create the BayesSearchCV object
    bayes_search = BayesSearchCV(pipe, param_dist, n_iter=25, scoring='r2', cv=tss, n_jobs=-1)

    # Fit the BayesSearchCV object to the data
    bayes_search.fit(X[:-252], y[:-252])

    # get the last train/test split indexes
    for _, test_index in tss.split(X):
        pass

    # Predicting the Test set results
    df_results['xgb_y_pred'] = bayes_search.best_estimator_.predict(X.loc[list(test_index)])

    print_scores(bayes_search, test_index)
    if verbose:
        tuning_plots(bayes_search, param_dist)
        display(pd.DataFrame(bayes_search.cv_results_))
    save_objet(bayes_search, 'saved_models/fitted_pipe_dtr.joblib')

    # store feature importance
    # loop over the features used in the best model and add the feature importance to the dataframe
    for i, feature in enumerate(bayes_search.best_estimator_.named_steps['feature_selection'].get_support(indices=True)):
        df_feature_importances.loc[df_feature_importances.index[feature], 'xgb3'] = bayes_search.best_estimator_.named_steps['regression'].feature_importances_[i]
    df_feature_importances.fillna(0, inplace=True)

    return df_results

df_results = XGBoost(X, y, df_results, df_feature_importances)

# Best parameters:  {'feature_selection__estimator__n_estimators': 10, 'feature_selection__threshold': '1*mean', 'regression__learning_rate': 0.3, 'regression__max_depth': 1, 'regression__n_estimators': 17}
# Best Accuracy: -8.65 %
# Best Model R2 score: -0.1026
# Best parameters:  {'pca__n_components': 0.58, 'regression__learning_rate': 0.4, 'regression__max_depth': 1, 'regression__n_estimators': 10}
# Best Accuracy: -36.28 %
# Best Model R2 score: -0.1866
## with commodities, exchange rates, and bench (ibov)
# Best parameters:  OrderedDict([('pca__n_components', 0.8), ('regression__booster', 'gbtree'), ('regression__learning_rate', 0.07568800319787194), ('regression__max_depth', 1)])
# Best Accuracy: -2.19 %
# Best Model R2 score: -0.0005
# Object saved in saved_models/fitted_pipe_dtr.joblib

Best parameters:  OrderedDict([('pca__n_components', 0.8), ('regression__booster', 'gbtree'), ('regression__learning_rate', 0.07568800319787194), ('regression__max_depth', 1)])
Best Accuracy: -2.19 %
Best Model R2 score: -0.0005
Object saved in saved_models/fitted_pipe_dtr.joblib


In [15]:
# CatBoost
def catboost(X, y, df_results, df_feature_importances):

    X_train, X_test, y_train, y_test = X[:-252], X[-252:], y[:-252], y[-252:]

    # Training CatBoost on the Training set
    from catboost import CatBoostRegressor
    regressor = CatBoostRegressor()
    regressor.fit(X_train, y_train)

    # Predicting the Test set results
    df_results['cat_y_pred'] = regressor.predict(X_test)

    # Evaluating the Model Performance
    from sklearn.metrics import r2_score
    print("R2 score: {:.4f}".format(r2_score(y_test, df_results['cat_y_pred'])))

    # store feature importance
    df_feature_importances['cat'] = regressor.feature_importances_

    return df_results

df_results = catboost(X, y, df_results, df_feature_importances)

# R2 score: -0.0366

Learning rate set to 0.053189
0:	learn: 0.0270207	total: 256ms	remaining: 4m 16s
1:	learn: 0.0269688	total: 299ms	remaining: 2m 29s
2:	learn: 0.0269143	total: 357ms	remaining: 1m 58s
3:	learn: 0.0268807	total: 398ms	remaining: 1m 39s
4:	learn: 0.0268425	total: 449ms	remaining: 1m 29s
5:	learn: 0.0268018	total: 501ms	remaining: 1m 22s
6:	learn: 0.0267580	total: 549ms	remaining: 1m 17s
7:	learn: 0.0267080	total: 604ms	remaining: 1m 14s
8:	learn: 0.0266668	total: 654ms	remaining: 1m 11s
9:	learn: 0.0266175	total: 704ms	remaining: 1m 9s
10:	learn: 0.0265872	total: 754ms	remaining: 1m 7s
11:	learn: 0.0265450	total: 805ms	remaining: 1m 6s
12:	learn: 0.0265140	total: 854ms	remaining: 1m 4s
13:	learn: 0.0264734	total: 896ms	remaining: 1m 3s
14:	learn: 0.0264437	total: 944ms	remaining: 1m 2s
15:	learn: 0.0264057	total: 997ms	remaining: 1m 1s
16:	learn: 0.0263772	total: 1.04s	remaining: 1m
17:	learn: 0.0263388	total: 1.09s	remaining: 59.3s
18:	learn: 0.0263148	total: 1.18s	remaining: 1m
19:	lear

In [16]:
# DataFrame with different metrics of success for each model
df_success = pd.DataFrame(columns=['mlr', 'dtr', 'rfr', 'svr', 'xgb', 'cat'],
                            index=['Correct sign predictions over total number of predictions', 
                                    'Correct up predictions over total number of up predictions', 
                                    'Correct up predictions over total number of up predictions 0.3',
                                    'Correct up predictions over total number of up predictions 0.5',
                                    'Correct up predictions over total number of up predictions 0.7',
                                    'Correct up predictions over total number of up predictions 1',
                                    'Correct up predictions over total number of up predictions 1+',
                                    '(BENCHMARK) Ups over total moves (informs about the data, not the prediction)'])

def get_success_rate(df_results, model):
    number_of_op = []
    # get the ratio of the correct sign predictions to the total number of predictions
    same_sign = (np.sign(df_results[model + '_y_pred']) == np.sign(df_results['y_test']))
    number_of_op.append(np.where(same_sign, 1, 0).sum())
    success_rate = number_of_op[0] / df_results.shape[0]
    df_success[model][0] = "{:.2f} %".format(success_rate*100)
    # get the ratio of the correct up predictions to the total number of up predictions
    number_of_op.append(np.where((same_sign) & (df_results[model + '_y_pred'] > 0), 1, 0).sum())
    success_rate = number_of_op[1] / np.where(df_results[model + '_y_pred'] > 0, 1, 0).sum()
    df_success[model][1] = "{:.2f} %".format(success_rate*100)
    # get the ratio of the correct up predictions to the total number of up predictions when the up prediction is greater than 5%
    lst = [0, 0.3, 0.5, 0.7, 1, np.inf]
    for i in range(1, len(lst)):
        mean, std = df_results[model + '_y_pred'].abs().mean(), df_results[model + '_y_pred'].std()
        upper_bond, lower_bond = mean + lst[i] * std, mean + lst[i-1] * std
        above_lower = (df_results[model + '_y_pred'] > lower_bond)
        below_upper = (df_results[model + '_y_pred'] < upper_bond)
        number_of_op.append(np.where((above_lower) & (below_upper) & (same_sign), 1, 0).sum())
        success_rate = number_of_op[-1] / np.where(((above_lower) & (below_upper)), 1, 0).sum()
        df_success[model][1 + i] = "{:.2f} %".format(success_rate*100)
    # get the ratio of the ups to the total number of moves
    number_of_op.append(np.where(df_results['y_test'] > 0, 1, 0).sum())
    success_rate = number_of_op[1 + len(lst)] / df_results.shape[0]
    df_success[model][1 + len(lst)] = "{:.2f} %".format(success_rate*100)
    df_success[model + '_op'] = number_of_op

for model in df_success.columns:
    get_success_rate(df_results, model)
df_success

# DataFrame without DPO feature

,mlr,dtr,rfr,svr,xgb,cat,mlr_op,dtr_op,rfr_op,svr_op,xgb_op,cat_op
Correct sign predictions over total number of predictions,53.97 %,53.97 %,53.97 %,46.43 %,54.37 %,46.83 %,136,136,136,117,137,118
Correct up predictions over total number of up predictions,53.97 %,53.97 %,54.31 %,52.27 %,54.43 %,51.85 %,136,136,126,23,129,42
Correct up predictions over total number of up predictions 0.3,nan %,nan %,nan %,100.00 %,nan %,55.56 %,0,0,0,3,0,5
Correct up predictions over total number of up predictions 0.5,nan %,nan %,100.00 %,100.00 %,nan %,55.56 %,0,0,1,2,0,5
Correct up predictions over total number of up predictions 0.7,nan %,nan %,nan %,100.00 %,nan %,75.00 %,0,0,0,1,0,3
Correct up predictions over total number of up predictions 1,nan %,nan %,50.00 %,nan %,nan %,0.00 %,0,0,1,0,0,0
Correct up predictions over total number of up predictions 1+,nan %,42.86 %,61.90 %,nan %,33.33 %,100.00 %,0,9,13,0,1,1
"(BENCHMARK) Ups over total moves (informs about the data, not the prediction)",53.97 %,53.97 %,53.97 %,53.97 %,53.97 %,53.97 %,136,136,136,136,136,136
